In [53]:
import implicit
import lightfm
import pandas as pd 
import numpy as np
from implicit.bpr import BayesianPersonalizedRanking as BPR
from scipy import sparse
import orion_recommend
from orion_recommend.evaluate import metrics
import lightfm

In [8]:
users = pd.read_csv("users", index_col = 0)
items = pd.read_csv("items", index_col = 0)
interactions = pd.read_csv("interactions", index_col = 0)

In [10]:
train = np.load("new_fa_train_data.npy")
test = np.load("new_fa_test_data.npy")

In [19]:
ui_matrix = np.load("fa_ui_matrix.npy")

In [84]:
ua_matrix = np.load("new_fa_ua_matrix.npy")

In [85]:
ua_matrix.shape

(86317, 471)

### Using user-item matrix


In [64]:
ui_matrix.shape, train.shape, test.shape

((86317, 10392), (76163, 15), (19041, 15))

In [65]:
test_users = np.unique(test[:,0])
train_users = np.unique(train[:,0])

In [66]:
test_items = np.unique(test[:,3])
train_items = np.unique(train[:,3])

In [67]:
test_users.shape, train_users.shape

((18645,), (70196,))

In [68]:
test_items.shape, train_items.shape

((6452,), (9989,))

In [69]:
intersection_users = np.intersect1d(train_users,test_users)
intersection_items =  np.intersect1d(train_items,test_items)

In [70]:
ind_users = pd.Series(test_users).isin(intersection_users)
ind_items = pd.Series(test_items).isin(intersection_items)

In [71]:
test_users = test_users[~ind_users]
test_items = test_items[~ind_items]

In [72]:
test_users.shape[0] + train_users.shape[0] - ui_matrix.shape[0] == 0

True

In [73]:
test_items.shape[0] + train_items.shape[0] - ui_matrix.shape[1] == 0

True

In [74]:
ui_train = ui_matrix[train_users]
ui_test = ui_matrix[test_users]

In [75]:
iu_train = ui_matrix.T[train_items]
iu_test = ui_matrix.T[test_items]

In [76]:
ui_train.shape, ui_test.shape

((70196, 10392), (16121, 10392))

In [78]:
train_sparse = sparse.coo_matrix(ui_train)
test_sparse = sparse.coo_matrix(ui_test)

In [81]:
test_sparse.shape

(16121, 10392)

In [82]:
train_sparse.shape

(70196, 10392)

### User-Attribute

In [87]:
ua_matrix.shape, train.shape, test.shape

((86317, 471), (76163, 15), (19041, 15))

In [88]:
test_users = np.unique(test[:,0])
train_users = np.unique(train[:,0])

In [89]:
test_items = np.unique(test[:,3])
train_items = np.unique(train[:,3])

In [90]:
test_users.shape, train_users.shape

((18645,), (70196,))

In [91]:
test_items.shape, train_items.shape

((6452,), (9989,))

In [95]:
test_users.shape[0] + train_users.shape[0] - ua_matrix.shape[0] == 0

True

In [97]:
ua_train = ua_matrix[train_users]
ua_test = ua_matrix[test_users]

In [98]:
ua_train.shape, ua_test.shape

((70196, 471), (16121, 471))

In [99]:
train_sparse = sparse.coo_matrix(ua_train)
test_sparse = sparse.coo_matrix(ua_test)

In [100]:
test_sparse.shape

(16121, 471)

In [101]:
train_sparse.shape

(70196, 471)

In [107]:
from lightfm import cross_validation

In [112]:
train, test = cross_validation.random_train_test_split(sparse.coo_matrix(ui_matrix), test_percentage=0.2, random_state=None)

In [124]:
user_features = np.load("user_attributes.npy")

In [135]:
user_features = sparse.coo_matrix(user_features)

In [126]:
user_features.shape

(86317, 5609)

In [131]:
item_features = np.load("new_fa_ia_matrix.npy")

In [133]:
item_features = sparse.coo_matrix(item_features)

In [136]:
from lightfm import LightFM
from lightfm.evaluation import precision_at_k


# Instantiate and train the model
model = LightFM(loss='warp')
model.fit(train, user_features=user_features,
          item_features=item_features, epochs=30)

In [139]:
# Evaluate the trained model
test_precision = precision_at_k(model, test,train, user_features=user_features,
          item_features=item_features,  k=10)

In [140]:
test_precision.mean()

0.0028460124